# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
part1data=pd.read_csv("../WeatherPy/output_data/cities.csv")
part1data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,George Town,5.4112,100.3354,30.96,84,40,0.45,MY,1628567407
1,Zhigansk,66.7697,123.3711,21.52,41,98,8.76,RU,1628567625
2,Vila Franca do Campo,37.7167,-25.4333,20.23,82,36,1.25,PT,1628567626
3,Ponta do Sol,32.6667,-17.1000,21.51,79,27,1.04,PT,1628567626
4,Hobart,-42.8794,147.3294,19.03,49,75,1.79,AU,1628567486
...,...,...,...,...,...,...,...,...,...
560,Bendigo,-36.7667,144.2833,17.70,42,1,0.89,AU,1628567756
561,Tyrma,50.0833,132.1667,20.97,47,46,2.45,RU,1628567756
562,Coachella,33.6803,-116.1739,38.97,40,1,1.34,US,1628567470
563,Kachīry,53.0675,76.1036,20.37,75,90,6.43,KZ,1628567757


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = part1data[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = part1data["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Ideal Weather Specifications: 
#temp. greater than or than or equal to 73 and less than or equal to 82
#wind speed less than 10mph
#cloudiness less than 15%

ideal_weather = part1data.loc[(part1data["Max Temp"]<= 82) & (part1data["Cloudiness"]< 15) & (part1data["Wind Speed"]< 10)]

ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,Nikolskoye,59.7035,30.7861,16.12,87,8,1.69,RU,1628567627
8,Cape Town,-33.9258,18.4232,8.95,93,0,0.51,ZA,1628567243
18,Ribeira Grande,38.5167,-28.7000,21.96,84,0,1.73,PT,1628567445
19,Asău,46.4333,26.4000,17.51,90,5,1.33,RO,1628567516
23,Sinnamary,5.3833,-52.9500,25.50,88,13,4.58,GF,1628567366
...,...,...,...,...,...,...,...,...,...
550,Darhan,49.4867,105.9228,22.66,51,9,3.26,MN,1628567753
552,Chibombo,-14.6569,28.0706,13.28,42,0,2.17,ZM,1628567754
554,‘Ewa Beach,21.3156,-158.0072,29.65,57,1,5.14,US,1628567754
560,Bendigo,-36.7667,144.2833,17.70,42,1,0.89,AU,1628567756


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df=ideal_weather.loc[:,["City", "Country", "Lat", "Lng"]]

#add "Hotel Name" Column and display table
hotel_df["Hotel Name"]=""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
7,Nikolskoye,RU,59.7035,30.7861,
8,Cape Town,ZA,-33.9258,18.4232,
18,Ribeira Grande,PT,38.5167,-28.7000,
19,Asău,RO,46.4333,26.4000,
23,Sinnamary,GF,5.3833,-52.9500,


In [33]:
#base url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type": "hotel",  
    "keyword": "hotel",
    "radius": 5000,
    "key": g_key,}

#Print header 
print("Beginning Data Retrieval")
print("-----------------------------")

#loop through rows
for index, row in hotel_df.iterrows():
    
    #add latitude & longitude 
    lat=row["Lat"]
    lng=row["Lng"]
    
    params["location"]=f"{lat},{lng}"
    
    #Assemble URL and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
   
     # extract results
    results = response["results"]
    
     # Save Name
    try:
        print(f"Matching Result: {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # print error message and final message
    except (KeyError, IndexError):
        print("Hotel not found. Skipping...")
        
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")

Beginning Data Retrieval
-----------------------------
Retrieving Results for Index 7: Nikolskoye.
Hotel not found. Skipping...
Retrieving Results for Index 8: Cape Town.
Matching Result: The Silo Hotel.
Retrieving Results for Index 18: Ribeira Grande.
Matching Result: BELO CAMPO.
Retrieving Results for Index 19: Asău.
Matching Result: Pensiunea Sophia.
Retrieving Results for Index 23: Sinnamary.
Matching Result: L'oiseau de Paradis.
Retrieving Results for Index 24: Bredasdorp.
Matching Result: De Hoop Collection.
Retrieving Results for Index 25: Chicama.
Matching Result: Balihai Bungalows.
Retrieving Results for Index 30: Yulara.
Matching Result: Sails in the Desert.
Retrieving Results for Index 34: Albany.
Hotel not found. Skipping...
Retrieving Results for Index 38: Fort Stewart.
Matching Result: Holiday Inn Express & Suites Hinesville East - Fort Stewart, an IHG Hotel.
Retrieving Results for Index 39: Margate.
Matching Result: Botany Bay Hotel.
Retrieving Results for Index 41: Herm

Matching Result: Isa Hotel.
Retrieving Results for Index 361: Marrakesh.
Matching Result: La Mamounia.
Retrieving Results for Index 362: Chernyshevskiy.
Matching Result: Sberbank.
Retrieving Results for Index 364: Neuquén.
Matching Result: Casino Magic Hotel & Casino.
Retrieving Results for Index 366: Ekhabi.
Hotel not found. Skipping...
Retrieving Results for Index 368: Alyangula.
Matching Result: Groote Eylandt Lodge.
Retrieving Results for Index 369: Buraidah.
Matching Result: Best Western Plus Buraidah.
Retrieving Results for Index 370: Kieta.
Hotel not found. Skipping...
Retrieving Results for Index 371: Uvalde.
Hotel not found. Skipping...
Retrieving Results for Index 378: Kingman.
Matching Result: La Quinta Inn & Suites by Wyndham Kingman.
Retrieving Results for Index 383: Lixouri.
Matching Result: Thalassa Boutique Hotel Kefalonia.
Retrieving Results for Index 386: Oktyabr'skoye.
Matching Result: Aleksandrovskiy.
Retrieving Results for Index 389: Beloha.
Hotel not found. Skippi

In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add marker layer of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
symbol_layer = gmaps.symbol_layer(locations, info_box_content=hotel_info)
fig.add_layer(symbol_layer)
# Display Map
fig